In [32]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup 
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException
from io import BytesIO
import datetime
from urllib import request,error
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
import math as m
from scipy.stats.kde import gaussian_kde
from numpy import linspace, hstack

In [7]:
def yahoo_download(url):
	url = "https://finance.yahoo.com/quote/" + url + "/history"
	head = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
	main_driver = webdriver.Chrome('/Users/lou_tun_chieh/Desktop/webdriver/chromedriver')  # 注意你們放CHROMEDRIVER的位置
	#告訴chromedriver 等下要找的element 如果沒有找到，要等10秒讓他們生完
	main_driver.implicitly_wait(10)
	main_driver.get(url)
	cookie_list = main_driver.get_cookies()
	cookies_dict = {}
	for cookie in cookie_list:
		cookies_dict['B'] = cookie['value']
	print(cookies_dict)

	date_btn_ele = main_driver.find_element_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/span[2]/span/input')
	date_btn_ele.click()
	main_driver.implicitly_wait(10)

	#點取date button
	date_btn_ele = main_driver.find_element_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/span[2]/div/input[1]').clear()
	date_btn_ele = main_driver.find_element_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/span[2]/div/input[1]').send_keys("01/01/2016")
	#date_btn_ele.click()
	main_driver.implicitly_wait(10)


	##點取done button
	done_btn_ele = main_driver.find_element_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/span[2]/div/div[3]/button[1]')
	done_btn_ele.click()
	main_driver.implicitly_wait(10)


	#點取frequency button
	fre_btn_ele = main_driver.find_element_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[3]/span/div/span/span')
	# print(fre_btn_ele.)
	fre_btn_ele.click()
	# main_driver.execute_script('arguments[0].innerHTML = "Weekly";', fre_btn_ele)
	fre_btn_ele = main_driver.find_element_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[3]/span/div[2]/div[3]').click()
	main_driver.implicitly_wait(10)

	

	#點取apply button
	done_btn_ele = main_driver.find_element_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/button')
	done_btn_ele.click()
	main_driver.implicitly_wait(10)
	time.sleep(20)
	
	#得到歷史資料的csv檔
	export_btn_ele = main_driver.find_element_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[2]/span[2]/a')
	csv_url = export_btn_ele.get_attribute('href')
	main_driver.quit()
	return csv_url, cookies_dict

In [8]:
def write_down_csv(etf_name):
	
	filename = "./ETF_month/" + etf_name + "_month.csv"
	if os.path.isfile(filename):
		df = pd.read_csv(filename)  
		df = pd.DataFrame(df)
		df = df.rename(columns={'Adj Close': etf_name})
		return df

	else:
		print(etf_name)
		tmp_url, tmp_cookie  = yahoo_download(etf_name)
		download = requests.get(tmp_url, cookies=tmp_cookie)
		df = pd.read_csv(BytesIO(download.content))
		#將該變數轉成pandas的dataframe格式
		df = pd.DataFrame(df)
		df = df.rename(columns={'Adj Close': etf_name})
		
		
		with open (filename, 'wb') as handle:
			for block in download.iter_content(1024):
				handle.write(block)
		
		return df

In [9]:
head = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
#必須帶有瀏覽器等資訊
etf_list_1 = "../Municipal Bond ETF List (29).csv"
etf_list_2 = "../Target Maturity Date Corporate Bond ETF List (24).csv"

df_1 = pd.read_csv(etf_list_1)
df_2 = pd.read_csv(etf_list_2)
df = pd.concat([df_1, df_2])

df = df[df["Inception"] < "2016"]
df = df.reset_index(drop=True)

tmp_df =  write_down_csv(df["Symbol"][0])
#Date = tmp_df["Date"]
result =  pd.concat([tmp_df["Date"], tmp_df[df["Symbol"][0]]], axis=1)
for i in range(1, len(df["Symbol"])):
	tmp_df =  write_down_csv(df["Symbol"][i])
	#print(tmp_df[df["Symbol"][i]])
	result =  pd.concat([result, tmp_df[df["Symbol"][i]]], axis=1)
	#print(result)

# result.to_csv('ETF_month.csv',encoding='utf_8_sig')

In [10]:
etf_month = pd.read_csv("ETF_month.csv")  
etf_month = pd.DataFrame(etf_month)
etf_month = etf_month.drop(['Unnamed: 0'], axis = 1)
etf_month = etf_month.drop(index = 0 )
etf_month = etf_month.drop(index = len(etf_month.index) )
etf_month.index = etf_month["Date"]
etf_month = etf_month.drop(["Date"], axis = 1 )
etf_month

                   MUB       VTEB        SHM        TFI         SUB  \
Date                                                                  
2016-01-01  103.421516  48.034252  47.060581  45.796093  102.589127   
2016-02-01  103.236122  48.118599  47.176022  45.833256  102.762848   
2016-03-01  103.713005  48.282925  46.981670  45.893768  102.506714   
2016-04-01  104.422119  48.706890  47.099037  46.319260  102.632256   
2016-05-01  104.695526  48.816494  47.047073  46.355389  102.550583   
2016-06-01  106.376984  49.656097  47.367340  47.357002  102.900040   
2016-07-01  106.328423  49.657806  47.519173  47.355648  103.179398   
2016-08-01  106.470253  49.673794  47.491222  47.457890  103.154877   
2016-09-01  105.861855  49.356052  47.256100  47.123882  102.764778   
2016-10-01  104.846016  48.980427  47.111008  46.549885  102.728607   
2016-11-01  100.551079  47.122952  46.279217  44.566578  101.541946   
2016-12-01  102.126884  47.721622  46.449089  44.792561  102.048988   
2017-0

In [11]:
etf_week = pd.read_csv("ETF_week.csv")  
etf_week = pd.DataFrame(etf_week)
etf_week = etf_week.drop(['Unnamed: 0'], axis = 1)
etf_week = etf_week.drop(index = 0 )
etf_week = etf_week.drop(index = len(etf_week.index) )
etf_week.index = etf_week["Date"]
etf_week = etf_week.drop(["Date"], axis = 1 )
etf_week

                   MUB       VTEB        SHM        TFI         SUB  \
Date                                                                  
2016-01-04  103.180519  48.127953  46.906670  45.740345  102.482964   
2016-01-11  103.226852  48.052971  47.002861  45.647408  102.338150   
2016-01-18  103.180519  48.062347  46.983624  45.758934  102.540840   
2016-01-25  103.421516  48.034252  47.060581  45.796093  102.589127   
2016-02-01  103.504974  48.231064  47.195259  45.870441  102.637383   
2016-02-08  103.917343  48.367378  47.289608  46.098412  102.757813   
2016-02-15  103.750175  48.292313  47.289608  46.098412  102.767456   
2016-02-22  103.452934  48.113995  47.174137  45.856556  102.738510   
2016-02-29  102.830521  47.926319  46.943176  45.540314  102.535675   
2016-03-07  103.091354  48.011490  47.022018  45.546501  102.487297   
2016-03-14  103.351929  48.161854  46.993130  45.658230  102.506645   
2016-03-21  103.361229  48.161854  46.983505  45.695473  102.516296   
2016-0

In [12]:
etf_week_return = etf_week/etf_week.shift(1)
etf_week_return = etf_week_return.drop(index = "2016-01-04" , axis = 0)
etf_week_return

                 MUB      VTEB       SHM       TFI       SUB       ITM  \
Date                                                                     
2016-01-11  1.000449  0.998442  1.002051  0.997968  0.998587  1.000413   
2016-01-18  0.999551  1.000195  0.999591  1.002443  1.001981  1.000413   
2016-01-25  1.002336  0.999415  1.001638  1.000812  1.000471  1.002476   
2016-02-01  1.000807  1.004097  1.002862  1.001623  1.000470  1.002470   
2016-02-08  1.003984  1.002826  1.001999  1.004970  1.001173  1.004407   
2016-02-15  0.998391  0.998448  1.000000  1.000000  1.000094  0.999590   
2016-02-22  0.997135  0.996308  0.997558  0.994753  0.999718  0.993443   
2016-02-29  0.993984  0.996099  0.995104  0.993104  0.998026  0.989274   
2016-03-07  1.002537  1.001777  1.001680  1.000136  0.999528  1.002988   
2016-03-14  1.002528  1.003132  0.999386  1.002453  1.000189  1.003748   
2016-03-21  1.000090  1.000000  0.999795  1.000816  1.000094  1.000000   
2016-03-28  1.002881  1.003317  0.9989

In [15]:
etf_month_return = etf_month/etf_month.shift(1)
etf_month_return = etf_month_return.drop(index = "2016-01-01" , axis = 0)
etf_month_return

                 MUB      VTEB       SHM       TFI       SUB       ITM  \
Date                                                                     
2016-02-01  0.998207  1.001756  1.002453  1.000811  1.001693  0.998353   
2016-03-01  1.004619  1.003415  0.995880  1.001320  0.997508  1.000699   
2016-04-01  1.006837  1.008781  1.002498  1.009271  1.001225  1.011662   
2016-05-01  1.002618  1.002250  0.998897  1.000780  0.999204  0.999486   
2016-06-01  1.016060  1.017199  1.006807  1.021607  1.003408  1.019867   
2016-07-01  0.999544  1.000034  1.003205  0.999971  1.002715  0.999831   
2016-08-01  1.001334  1.000322  0.999412  1.002159  0.999762  1.001330   
2016-09-01  0.994286  0.993603  0.995049  0.992962  0.996218  0.994097   
2016-10-01  0.990404  0.992389  0.996930  0.987819  0.999648  0.989517   
2016-11-01  0.959036  0.962077  0.982344  0.957394  0.988449  0.945213   
2016-12-01  1.015672  1.012704  1.003671  1.005071  1.004993  1.013658   
2017-01-01  1.005890  1.005612  1.0092

In [28]:
def compute_riskiness(data,i):
#    print data.var()/(2*data.mean())
    probDensityFun = gaussian_kde(data)
    x = linspace(-0.2,0.2,200)
    p=(probDensityFun(x)/probDensityFun(x).sum())
    g=list(map(lambda x: m.exp(-x*i), x))
    return 1-p.dot(g)

def compute_riskiness_r(data):
    if data.mean()<0:
        temp_past=1
        i=0
        while temp_past>=-1e-6 and compute_riskiness(data,i)>=-1e-6:
            temp_past=compute_riskiness(data,i)
            i=i-1
        i=i+1
        temp_past=1
        j=0
        while temp_past>=-1e-6 and compute_riskiness(data,i+j/10.0)>=-1e-6:
            temp_past=compute_riskiness(data,i+j/10.0)
            j=j-1
        j=j+1
        temp_past=1
        k=0
        while temp_past>=-1e-6 and compute_riskiness(data,i+j/10.0+k/100.0)>=-1e-6:
            temp_past=compute_riskiness(data,i+j/10.0+k/100.0)
            k=k-1
        k=k+1
        return i+j/10.0+k/100.0
    else:
        temp_past=1
        i=0
        while temp_past>=-1e-6 and compute_riskiness(data,i)>=-1e-6:
            temp_past=compute_riskiness(data,i)
            i=i+1
        i=i-1
        temp_past=1
        j=0
        while temp_past>=-1e-6 and compute_riskiness(data,i+j/10.0)>=-1e-6:
            temp_past=compute_riskiness(data,i+j/10.0)
            j=j+1
        j=j-1
        temp_past=1
        k=0
        while temp_past>=-1e-6 and compute_riskiness(data,i+j/10.0+k/100.0)>=-1e-6:
            temp_past=compute_riskiness(data,i+j/10.0+k/100.0)
            k=k+1
        k=k-1
        return i+j/10.0+k/100.0

In [53]:
#riskiness_combine_week=pd.DataFrame()
riskiness_combine_week = []
columns_name = []
for k in range(etf_week_return.shape[1]): # for all ETF
    #print(k)
    temp_etf=etf_week_return.iloc[:,k].dropna()
    columns_name.append(etf_week_return.columns[k])
    riskiness = []
    riskiness.append(compute_riskiness_r(temp_etf.iloc[0:156]-1))
    #print(riskiness)
    riskiness = m.exp(-riskiness[0])
    riskiness_combine_week.append(riskiness)

riskiness_combine_week_list = riskiness_combine_week
    
    #riskiness_combine_week=pd.concat([riskiness_combine_week,riskiness],axis=1, sort=True)


#     if len(temp_etf) >= 156:
#         columns_name.append(etf_data_week_return.columns[k])
#         for i in range(156, len(temp_etf)+1):
#             riskiness.append(compute_riskiness_r(temp_etf.iloc[i-156:i]-1))
#         riskiness = pd.DataFrame(riskiness,index=temp_etf.index[155:])
#         riskiness = riskiness[0].map(lambda x: m.exp(-x))
#         riskiness = pd.DataFrame(riskiness,index=temp_etf.index[155:])
#         riskiness_combine_week=pd.concat([riskiness_combine_week,riskiness],axis=1)
# riskiness_combine_week.columns=columns_name

0
[32.63]
1
[36.68]
2
[61.39]
3
[21.9]
4
[204.55]
5
[0.56]
6
[27.99]
7
[60.64]
8
[33.99]
9
[0.0]
10
[166.73]
11
[32.55]
12
[14.27]
13
[45.6]
14
[335.26]
15
[17.12]
16
[35.309999999999995]
17
[131.09]
18
[262.28999999999996]
19
[81.59]
20
[63.44]
21
[98.1]
22
[63.730000000000004]
23
[164.95000000000002]
24
[51.17]
25
[59.89]
26
[47.690000000000005]
27
[34.9]
28
[37.620000000000005]
29
[29.54]
30
[30.07]
31
[114.6]
32
[102.22]
33
[35.17]
34
[43.24]


In [67]:
riskiness_combine_week = riskiness_combine_week_list
length = len(riskiness_combine_week)
riskiness_combine_week = pd.DataFrame(np.array(riskiness_combine_week).reshape(-1, 1), index = columns_name, columns = ['riskiness'])
riskiness_combine_week = riskiness_combine_week.sort_values(['riskiness'], ascending=True)
ranking = [i for i in range(1, length+1)]
riskiness_combine_week['Ranking'] = ranking
riskiness_combine_week = riskiness_combine_week.drop('riskiness', 1)
riskiness_combine_week

,Ranking
PVI,1
BSCJ,2
SUB,3
MEAR,4
IBDK,5
BSCK,6
IBDC,7
IBCD,8
IBDL,9
BSCL,10


In [68]:
riskiness_combine_week.to_csv("riskiness_combine_week.csv")

In [69]:
riskiness_combine_month = []
columns_name = []
for k in range(etf_month_return.shape[1]): # for all ETF
    #print(k)
    temp_etf=etf_month_return.iloc[:,k].dropna()
    columns_name.append(etf_month_return.columns[k])
    riskiness = []
    riskiness.append(compute_riskiness_r(temp_etf.iloc[0:36]-1))
    #print(riskiness)
    riskiness = m.exp(-riskiness[0])
    riskiness_combine_month.append(riskiness)

riskiness_combine_month_list = riskiness_combine_month
riskiness_combine_month

0
[19.53]
1
[25.5]
2
[29.97]
3
[16.62]
4
[72.36]
5
[12.7]
6
[21.62]
7
[30.52]
8
[30.69]
9
[19.979999999999997]
10
[328.58]
11
[14.31]
12
[8.239999999999998]
13
[47.160000000000004]
14
[931.8199999999999]
15
[11.129999999999999]
16
[38.93]
17
[90.26]
18
[236.72]
19
[72.23]
20
[57.660000000000004]
21
[103.53]
22
[65.61]
23
[226.88000000000002]
24
[50.37]
25
[55.37]
26
[45.82]
27
[39.690000000000005]
28
[42.28]
29
[32.48]
30
[30.59]
31
[226.82000000000002]
32
[116.81]
33
[52.349999999999994]
34
[51.09]


[3.297833827230616e-09,
 8.423463754468647e-12,
 9.642605015268938e-14,
 6.053767127397022e-08,
 3.753628513701758e-32,
 3.0511255580364225e-06,
 4.0789962049207007e-10,
 5.56329913513334e-14,
 4.693559744513081e-14,
 2.1027916876128252e-09,
 1.9930543237777028e-143,
 6.098822080395936e-07,
 0.0002638842494547184,
 3.3012030177556055e-21,
 0.0,
 1.4665687152517056e-05,
 1.2385564904517142e-17,
 6.3180064278222956e-40,
 1.5624648062686812e-103,
 4.274738691860157e-32,
 9.09034096916597e-26,
 1.0901651349628999e-45,
 3.205819323395001e-29,
 2.932702070478368e-99,
 1.3322537352002598e-22,
 8.976655057112995e-25,
 1.2607437944855337e-20,
 5.79231288539483e-18,
 4.345395448490776e-19,
 7.836375312862996e-15,
 5.1871857318864316e-14,
 3.114050238563183e-99,
 1.8623511389778008e-51,
 1.839432571236886e-23,
 6.484775111198212e-23]

In [90]:
riskiness_combine_month = riskiness_combine_month_list
length = len(riskiness_combine_month)
riskiness_combine_month = pd.DataFrame(np.array(riskiness_combine_month).reshape(-1, 1), index = columns_name, columns = ['riskiness'])
riskiness_combine_month = riskiness_combine_month.sort_values(['riskiness'], ascending=True)
ranking = [i for i in range(1, length+1)]
riskiness_combine_month['Ranking'] = ranking
riskiness_combine_month = riskiness_combine_month.drop('riskiness', 1)
riskiness_combine_month.to_csv("riskiness_combine_month.csv")
riskiness_combine_month


,Ranking
PVI,1
MEAR,2
BSCJ,3
IBDK,4
IBDC,5
IBCD,6
IBDL,7
BSCK,8
SUB,9
BSCL,10


In [85]:
#omega_combine_week=pd.DataFrame()
bond_rate=pd.read_excel("USG3M.xlsx",index_col=0,parse_dates=True,header=None)
USG3M=bond_rate[1]
omega_combine_week = []
columns_name = []
for k in range(etf_week_return.shape[1]):
    #print(k)
    temp_etf = etf_week_return.iloc[:,k].dropna()
    columns_name.append(etf_week_return.columns[k])
    omega = []
    probDensityFun = gaussian_kde(temp_etf.iloc[0:156]-1)
    x = np.linspace(-0.5,0.5,10000)
    L = USG3M.loc[temp_etf.iloc[0:156].index[-1]]
    p = (probDensityFun(x)/probDensityFun(x).sum())
    p = p.cumsum()
    p = pd.DataFrame(p, index=x)
    omega = ((1-p).loc[L/100/52:].sum())/(p.loc[:L/100/52].sum())
    omega = omega.loc[0]
    print(omega)
    omega_combine_week.append(omega)

omega_combine_week_list = omega_combine_week
omega_combine_week

0.8619926067012126
0.8720127858585578
0.5992592437698039
0.8767959942456114
0.5020107523620275
0.8797103123171499
0.9553545051518076
1.0031511271515174
0.8470232406949927
0.9932499676904187
0.5503368539559709
0.9285283588482879
0.9372302585904879
0.7297773307139579
0.3622735249833677
0.9580642659942201
0.7184112369396844
0.8890799836593025
0.8048840264226818
0.9581062072533157
0.9930627724266171
0.8596952899175158
0.9040470327766332
0.8007436216710325
0.9508796129863811
1.014483047763856
0.9831850111313003
0.9922544233138846
1.0192302973317258
0.9959141014700527
0.9801025905702894
1.028513780169712
0.9677569702434068
1.0828362132477436
1.0537884202096761


[0.8619926067012126,
 0.8720127858585578,
 0.5992592437698039,
 0.8767959942456114,
 0.5020107523620275,
 0.8797103123171499,
 0.9553545051518076,
 1.0031511271515174,
 0.8470232406949927,
 0.9932499676904187,
 0.5503368539559709,
 0.9285283588482879,
 0.9372302585904879,
 0.7297773307139579,
 0.3622735249833677,
 0.9580642659942201,
 0.7184112369396844,
 0.8890799836593025,
 0.8048840264226818,
 0.9581062072533157,
 0.9930627724266171,
 0.8596952899175158,
 0.9040470327766332,
 0.8007436216710325,
 0.9508796129863811,
 1.014483047763856,
 0.9831850111313003,
 0.9922544233138846,
 1.0192302973317258,
 0.9959141014700527,
 0.9801025905702894,
 1.028513780169712,
 0.9677569702434068,
 1.0828362132477436,
 1.0537884202096761]

In [86]:
omega_combine_week = omega_combine_week_list
#print(omega_combine_week)
length = len(omega_combine_week)
omega_combine_week = pd.DataFrame(np.array(omega_combine_week).reshape(-1, 1), index = columns_name, columns = ['omega'])
#print(omega_combine_week)
omega_combine_week = omega_combine_week.sort_values(['omega'], ascending=False)
ranking = [i for i in range(1, length+1)]
omega_combine_week['Ranking'] = ranking
omega_combine_week = omega_combine_week.drop('omega', 1)
omega_combine_week.to_csv("omega_combine_week.csv")
omega_combine_week

[0.8619926067012126, 0.8720127858585578, 0.5992592437698039, 0.8767959942456114, 0.5020107523620275, 0.8797103123171499, 0.9553545051518076, 1.0031511271515174, 0.8470232406949927, 0.9932499676904187, 0.5503368539559709, 0.9285283588482879, 0.9372302585904879, 0.7297773307139579, 0.3622735249833677, 0.9580642659942201, 0.7184112369396844, 0.8890799836593025, 0.8048840264226818, 0.9581062072533157, 0.9930627724266171, 0.8596952899175158, 0.9040470327766332, 0.8007436216710325, 0.9508796129863811, 1.014483047763856, 0.9831850111313003, 0.9922544233138846, 1.0192302973317258, 0.9959141014700527, 0.9801025905702894, 1.028513780169712, 0.9677569702434068, 1.0828362132477436, 1.0537884202096761]
         omega
MUB   0.861993
VTEB  0.872013
SHM   0.599259
TFI   0.876796
SUB   0.502011
ITM   0.879710
PZA   0.955355
FMB   1.003151
MUNI  0.847023
SMB   0.993250
MEAR  0.550337
SHYD  0.928528
MLN   0.937230
SMMU  0.729777
PVI   0.362274
RVNU  0.958064
PRB   0.718411
BSCK  0.889080
BSCJ  0.804884
B

,Ranking
IBDD,1
IBCE,2
IBDC,3
BSCO,4
BSCN,5
FMB,6
IBDQ,7
SMB,8
BSCM,9
IBDP,10


In [87]:
omega_combine_month = []
columns_name = []
for k in range(etf_month_return.shape[1]):
    #print(k)
    temp_etf = etf_month_return.iloc[:,k].dropna()
    columns_name.append(etf_month_return.columns[k])
    omega = []
    probDensityFun = gaussian_kde(temp_etf.iloc[0:36]-1)
    x = np.linspace(-0.5,0.5,10000)
    L = USG3M.loc[temp_etf.iloc[0:36].index[-1]]
    p = (probDensityFun(x)/probDensityFun(x).sum())
    p = p.cumsum()
    p = pd.DataFrame(p, index=x)
    omega = ((1-p).loc[L/100/12:].sum())/(p.loc[:L/100/12].sum())
    omega = omega.loc[0]
    print(omega)
    omega_combine_month.append(omega)

omega_combine_month_list = omega_combine_month
omega_combine_month

0.8502248173069975
0.9009205405464606
0.5292605727839571
0.872413670669498
0.41966222810583104
0.8976785874884824
1.0006559648983924
1.0855272602404131
0.8387704487188773
0.6539863308187917
0.22031350887888676
0.9547575987039946
0.9464300775062273
0.5742356331349703
0.035226076150046935
1.0140104067234705
0.5605851418499967
0.9742690711956212
0.8161165719572081
1.1005210131351189
1.2072191237716652
0.8701881637484739
1.0213985724160093
0.8101437413788569
1.1162721711450931
1.2958241698076098
1.2057455797181236
1.244168178236301
1.2801816161906534
1.2187985108002395
1.1854674790917163
1.3708330518781537
1.163063562594103
1.4278893375546955
1.3383294317547505


[0.8502248173069975,
 0.9009205405464606,
 0.5292605727839571,
 0.872413670669498,
 0.41966222810583104,
 0.8976785874884824,
 1.0006559648983924,
 1.0855272602404131,
 0.8387704487188773,
 0.6539863308187917,
 0.22031350887888676,
 0.9547575987039946,
 0.9464300775062273,
 0.5742356331349703,
 0.035226076150046935,
 1.0140104067234705,
 0.5605851418499967,
 0.9742690711956212,
 0.8161165719572081,
 1.1005210131351189,
 1.2072191237716652,
 0.8701881637484739,
 1.0213985724160093,
 0.8101437413788569,
 1.1162721711450931,
 1.2958241698076098,
 1.2057455797181236,
 1.244168178236301,
 1.2801816161906534,
 1.2187985108002395,
 1.1854674790917163,
 1.3708330518781537,
 1.163063562594103,
 1.4278893375546955,
 1.3383294317547505]

In [91]:
omega_combine_month = omega_combine_month_list
#print(omega_combine_week)
length = len(omega_combine_month)
omega_combine_month = pd.DataFrame(np.array(omega_combine_month).reshape(-1, 1), index = columns_name, columns = ['omega'])
#print(omega_combine_week)
omega_combine_month = omega_combine_month.sort_values(['omega'], ascending=False)
ranking = [i for i in range(1, length+1)]
omega_combine_month['Ranking'] = ranking
omega_combine_month = omega_combine_month.drop('omega', 1)
omega_combine_month.to_csv("omega_combine_month.csv")
omega_combine_month

,Ranking
IBDD,1
IBDC,2
IBCE,3
BSCN,4
BSCO,5
IBDP,6
IBDQ,7
BSCM,8
IBDO,9
BSCP,10


In [94]:
ASKSR_week = pd.DataFrame(np.arange(len(etf_month_return.columns)).reshape((len(etf_month_return.columns), 1)))
adj_list = []
cumprod_return_list = []
sharpe_week = []
for i in range(len(etf_week_return.columns)):
	tmp_etf = etf_week_return.iloc[:, i]
	tmp_etf = tmp_etf - 1
	# tmp_etf = np.log(tmp_etf)

	mean = tmp_etf.mean()
	std  = tmp_etf.std()
	skew = tmp_etf.skew()
	kurt = tmp_etf.kurt()
	var =  -1.96


	Z = var + ( skew * ((var**2) - 1)/6) + (kurt * ((var**3) - 3 * var)/24)-((skew**2) * (2 * (var**3)- 5 * var)/36)
	# print(mean, std, skew, kurt,Z, mean/(std/m.sqrt(7)))
	sharpe_week.append( mean/(std/m.sqrt(7)))
	adj = (-0.5) * (std**2) * 52 - Z * std * m.sqrt(len(etf_week_return.index))
	cumprod_return = (np.exp(tmp_etf).cumprod()[-1])
	adj_list.append(adj)	
	cumprod_return_list.append(cumprod_return)


ASKSR_week.columns = ["Ranking"]
ASKSR_week.index = etf_month_return.columns
for i in range (len(etf_week_return.columns)):
	ASKSR_week.iloc[i,0] = cumprod_return_list[i]/adj_list[i]#sharpe_week[i]

ASKSR_week = ASKSR_week.sort_values(by = ['Ranking'], ascending=False)
ranking = [i for i in range(1, 36)]
ASKSR_week["Ranking"] = ranking 
ASKSR_week

,Ranking
PVI,1
MEAR,2
SUB,3
BSCJ,4
IBDK,5
SHM,6
BSCK,7
IBDC,8
IBCD,9
IBDL,10


In [95]:
ASKSR_month = pd.DataFrame(np.arange(len(etf_month_return.columns)).reshape((len(etf_month_return.columns), 1)))
adj_list = []
cumprod_return_list = []
sharpe_month = []
for i in range(len(etf_month_return.columns)):
	tmp_etf = etf_month_return.iloc[:, i]
	tmp_etf = tmp_etf
	tmp_etf = np.log(tmp_etf)

	mean = tmp_etf.mean()
	std  = tmp_etf.std()
	skew = tmp_etf.skew()
	kurt = tmp_etf.kurt()
	var = -1.96

	Z = var + ( skew * ((var**2) - 1)/6) + (kurt * ((var**3) - 3 * var)/24)-((skew**2) * (2 * (var**3)- 5 * var)/36)
		# print(mean, std, skew, kurt,Z)
	sharpe_month.append( mean/(std/m.sqrt(30)))
	adj = (-0.5) * (std**2) * 52 - Z * std * m.sqrt(len(etf_week_return.index))
	cumprod_return = (np.exp(tmp_etf).cumprod()[-1])
	adj_list.append(adj)	
	cumprod_return_list.append(cumprod_return)

ASKSR_month.index = etf_month_return.columns
ASKSR_month.columns = ['Ranking']
for i in range (len(etf_month_return.columns)):
	ASKSR_month.iloc[i,0] = cumprod_return_list[i]/adj_list[i]  #sharpe_month[i]

ASKSR_month = ASKSR_month.sort_values(by = ['Ranking'], ascending=False)
ranking = [i for i in range(1, 36)]
ASKSR_month["Ranking"] = ranking 

ASKSR_month

,Ranking
PVI,1
MEAR,2
BSCJ,3
IBDC,4
IBDK,5
IBCD,6
IBDL,7
SUB,8
BSCK,9
IBDM,10
